In [ ]:
# importing useful libraries
import numpy as np
import tensorflow as tf
import random as python_random

#setting random seed
np.random.seed(1)
python_random.seed(12)
tf.random.set_seed(123)

import pandas as pd

from sklearn.metrics import fbeta_score

from skimage import io
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Metric

%matplotlib inline

In [ ]:
train_classes_df = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
n_train = !ls ../input/planets-dataset/planet/planet/train-jpg | wc -l
print('There are {} images in the training set'.format(n_train[0]))
train_classes_df.head()

In [ ]:
image_number =10
img = io.imread('../input/planets-dataset/planet/planet/train-jpg/train_{}.jpg'.format(image_number))
print('images are of shape {}'.format(img.shape))
plt.imshow(img)
plt.show()

In [ ]:
# finding the unique labels in our dataset
unique_labels = set()

# defining a function to add labels to a set
def append_labels(tags):
    for tag in tags.split():
        unique_labels.add(tag)

train_classes = train_classes_df.copy() # creates a copy to avoid mutating traiin_classes_df
train_classes['tags'].apply(append_labels)
unique_labels = list(unique_labels) # converting unique_labels to a list so they can be ordered
print('Unique labels are {} in number and they are {}'.format(len(unique_labels), unique_labels))

# let's do one hot encoding on the labels in 'train_classes'
for tag in unique_labels:
    train_classes[tag] = train_classes['tags'].apply(lambda x: 1 if tag in x.split() else 0)
    
# adding '.jpg' extension to 'image_name'
train_classes['image_name'] = train_classes['image_name'].apply(lambda x: '{}.jpg'.format(x)) 
train_classes.head(3)

In [ ]:
y_col = list(train_classes.columns[2:]) # storing the tags column names as a variable

# initializing an image generator and rescaling the images
image_gen = ImageDataGenerator(rescale=1/255)

# loading images from dataframe
X = image_gen.flow_from_dataframe(dataframe=train_classes, \
        directory='/kaggle/input/planets-dataset/planet/planet/train-jpg/', x_col='image_name', y_col=y_col, \
       target_size=(128, 128), class_mode='raw', seed=1, batch_size=128)

# X is an iterable, It contains 317 batches, each batch contains 128 images and labels because 
#40479 / 128 is 316 remainder 31 each image is of shape (128, 128, 3), each label is of shape (17, )

# let's abitrarily view an image say the 109th image
x109 = X[0][0][109] # first batch, images, 109th image
y109 = X[0][1][109] # first batch, labels, 109th label
print("Each image's shape is {}".format(x109.shape))
print("Each label's shape is {}".format(y109.shape))
print('We have {} batches'.format(len(X)))
print('Each batch has {} images/labels'.format(X[0][0].shape[0]))
print('40479/128 is {:.2F}, so the last batch will have {} images/labels'.format(\
                                                                                40479/128, X[316][0].shape[0]))

In [ ]:
beta = 2 # arbitrarily setting beta to 2. You can set it to any value you choose to
threshold = 0.2 # arbitrarily setting beta to 0.2. You can set it to any value you choose to

def multi_label_fbeta(ytrue , ypred, beta=beta, average='samples', threshold=threshold, epsilon=1e-7, \
                      sample_weight=None):
    # epsilon is set to avoid division by zero error
    beta_squared = beta**2

    # casting ytrue and ypred as floats
    ytrue = tf.cast(ytrue, tf.float32)
    
    # making ypred one hot encoded 
    ypred = tf.cast(tf.greater_equal(tf.cast(ypred, tf.float32), tf.constant(threshold)), tf.float32)
    
    if average == 'samples':
        tp = tf.reduce_sum(ytrue * ypred, axis=-1) # calculating true positives
        predicted_positive = tf.reduce_sum(ypred, axis=-1) # calculating predicted positives
        actual_positive = tf.reduce_sum(ytrue, axis=-1) # calculating actual positives
    
    else: # either any of 'macro', 'weighted' and 'raw'
        tp = tf.reduce_sum(ytrue * ypred, axis=0) # calculating true positives
        predicted_positive = tf.reduce_sum(ypred, axis=0) # calculating predicted positives
        actual_positive = tf.reduce_sum(ytrue, axis=0) # calculating actual positives
    
    # calculating precision and recall
    precision = tp/(predicted_positive+epsilon)
    recall = tp/(actual_positive+epsilon)

    # finding fbeta
    fb = (1+beta_squared)*precision*recall / (beta_squared*precision + recall + epsilon)

    if average == 'weighted':
        supports = tf.reduce_sum(ytrue, axis=0)
        return tf.reduce_sum(fb*supports / tf.reduce_sum(supports))

    elif average == 'raw':
        return fb
    
    elif average == 'samples' and sample_weight is not None:
        return tf.reduce_sum(fb*sample_weight)
    
    return tf.reduce_mean(fb) # then it is either 'macro' or 'samples' (without sample weight)

In [ ]:
def build_model(metrics=multi_label_fbeta):
    model = Sequential() # initializes a sequential layer
    
    model.add(Conv2D(filters=128, kernel_size=3, input_shape=(128, 128, 3))) # adds a convolutional layer
    
    model.add(Flatten()) # flattens the layer
    
    model.add(Dense(17, activation='sigmoid')) # output layer. Notice the actiation function
    
    opt = Adam(lr=1e-2) # initializes an optimizer
    
    # compiling the model
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[metrics])
    
    return model

In [ ]:
train_image_gen = ImageDataGenerator(rescale=1/255,validation_split=0.2)

# generating the 80% training image data
train_gen = train_image_gen.flow_from_dataframe(dataframe=train_classes, \
        directory='../input/planets-dataset/planet/planet/train-jpg/', x_col='image_name', y_col=y_col, \
       target_size=(128, 128), class_mode='raw', seed=0, batch_size=128, subset='training')

# generating the 20% validation image data
val_gen = train_image_gen.flow_from_dataframe(dataframe=train_classes, \
        directory='../input/planets-dataset/planet/planet/train-jpg/', x_col='image_name', y_col=y_col, \
       target_size=(128, 128), class_mode='raw', seed=0, batch_size=128, subset='validation')

# setting step size for training and validation image data
step_train_size = int(np.ceil(train_gen.samples / train_gen.batch_size))
step_val_size = int(np.ceil(val_gen.samples / train_gen.batch_size))

In [ ]:
model = build_model() # building model for training

# fitting the model
model.fit(x=train_gen, steps_per_epoch=step_train_size, validation_data=val_gen, validation_steps=step_val_size,
         epochs=3)

In [ ]:
random_ytrue = np.random.choice([0, 1], (20, 17))
random_ypred = np.random.choice([0, 1], (20, 17))

print('f1_score of prediction using multi_label_fbeta is {}'.format(multi_label_fbeta(random_ytrue, random_ypred)))
print('f1_score of prediction using scikit-learn fbeta is {}'.format(fbeta_score(\
                                                    random_ytrue, random_ypred, beta=2, average='samples')))

In [ ]:
n_class = 17
class StatefullMultiLabelFBeta(Metric):
    def __init__(self, name='state_full_binary_fbeta', beta=beta, average='samples', \
                 n_class=n_class, threshold=threshold, epsilon=1e-7, **kwargs):
        
        # initializing an object of the super class
        super(StatefullMultiLabelFBeta, self).__init__(name=name, **kwargs)
            
        # initializing atrributes
        self.tp = self.add_weight(name='tp', shape=(n_class,), initializer='zeros') # initializing true positives
        self.actual_positives = self.add_weight(name='ap', shape=(n_class,), initializer='zeros') 
        self.predicted_positives = self.add_weight(name='pp', shape=(n_class,), initializer='zeros')

        self.n_samples = self.add_weight(name='n_samples', initializer='zeros')
        self.sum_fb = self.add_weight(name='sum_fb', initializer='zeros')

        # initializing other atrributes that wouldn't be changed for every object of this class
        self.beta_squared = beta**2
        self.average = average
        self.n_class = n_class
        self.threshold = threshold
        self.epsilon = epsilon

    def update_state(self, ytrue, ypred, sample_weight=None):
        # casting ytrue float dtype
        ytrue = tf.cast(ytrue, tf.float32)
        
        # making ypred one hot encoded 
        ypred = tf.cast(tf.greater_equal(tf.cast(ypred, tf.float32), tf.constant(threshold)), tf.float32)
        
        if self.average == 'samples': # we are to keep track of only fbeta
            # calculate true positives, predicted positives and actual positives atrribute along the last axis
            tp = tf.reduce_sum(ytrue*ypred, axis=-1) 
            predicted_positives = tf.reduce_sum(ypred, axis=-1)
            actual_positives = tf.reduce_sum(ytrue, axis=-1)
            
            precision = tp/(predicted_positives+self.epsilon) # calculate the precision
            recall = tp/(actual_positives+self.epsilon) # calculate the recall
            
            # calculate the fbeta score
            fb = (1+self.beta_squared)*precision*recall / (self.beta_squared*precision + \
                                                                      recall + self.epsilon)
            
            if sample_weight is not None: # if sample weight is available for stand alone usage
                self.fb = tf.reduce_sum(fb*sample_weight)
            else:
                n_rows = tf.reduce_sum(tf.shape(ytrue)*tf.constant([1, 0])) # getting the number of rows in ytrue
                self.n_samples.assign_add(tf.cast(n_rows, tf.float32)) # updating n_samples
                self.sum_fb.assign_add(tf.reduce_sum(fb)) # getting the running sum of fb
                self.fb = self.sum_fb / self.n_samples # getting the running mean of fb

        else:
            # keep track of true, predicted and actual positives because they are calculated along axis 0
            self.tp.assign_add(tf.reduce_sum(ytrue*ypred, axis=0)) 
            self.assign_add(predicted_positives = tf.reduce_sum(ypred, axis=0))
            self.actual_positives.assign_add(tf.reduce_sum(ytrue, axis=0)) 
            
    def result(self):
        if self.average != 'samples':
            precision = self.tp/(self.predicted_positives+self.epsilon) # calculate the precision
            recall = self.tp/(self.actual_positives+self.epsilon) # calculate the recall

            # calculate the fbeta score
            fb = (1+self.beta_squared)*precision*recall / (self.beta_squared*precision + \
                                                                      recall + self.epsilon)
            if self.average == 'weighted':
                return tf.reduce_sum(fb*self.actual_positives / tf.reduce_sum(self.actual_positives))

            elif self.average == 'raw':
                return fb
            
            return tf.reduce_mean(fb) # then it is 'macro' averaging 
    
        return self.fb # then it is either 'samples' with or without sample weight

    def reset_states(self):
        self.tp.assign(tf.zeros(self.n_class)) # resets true positives to zero
        self.predicted_positives.assign(tf.zeros(self.n_class)) # resets predicted positives to zero
        self.actual_positives.assign(tf.zeros(self.n_class)) # resets actual positives to zero
        self.n_samples.assign(0)
        self.sum_fb.assign(0)

In [ ]:
statefull_multi_label_fbeta = StatefullMultiLabelFBeta()

statefull_model = build_model(metrics=statefull_multi_label_fbeta) # building model for training

train_gen.reset() # reseting the training set generator 
val_gen.reset() # reseting the validation set generator 

# fitting the model
statefull_model.fit(x=train_gen, steps_per_epoch=step_train_size, validation_data=val_gen, \
                    validation_steps=step_val_size, epochs=3)

In [ ]:
n_sample = 20
n_class = 17
m = StatefullMultiLabelFBeta(n_class=n_class) # initializes a stateful multi class fbeta object

random_ytrue = np.random.choice([0, 1], (n_sample, n_class))
random_ypred = np.random.choice([0, 1], (n_sample, n_class))

m.update_state(random_ytrue, random_ypred)
print('Intermediate result for stateful multi class fbeta is: {}'.format(float(m.result())))
print('Intermediate result for scikit-learn fbeta is: {}'.format(fbeta_score(\
                                                        random_ytrue, random_ypred, beta=2, average='samples')))
print()

increment_size = 20
a_true = np.random.choice([0, 1], (increment_size, n_class))
a_pred = np.random.choice([0, 1], (increment_size, n_class))

m.update_state(a_true, a_pred)
print('Final result for stateful multi class fbeta is: {}'.format(float(m.result())))

arr_true = np.append(random_ytrue, a_true, axis=0)
arr_pred = np.append(random_ypred, a_pred, axis=0)

print('Final result for scikit-learn multi class fbeta is: {}'.format(\
                                                    fbeta_score(arr_true, arr_pred, beta=2, average='samples')))